In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

Using TensorFlow backend.


In [ ]:
# specifying data directory
dir_data = "D:\\Great Learning\\R7"
dir_train = os.path.join(dir_data,"train")
dir_test = os.path.join(dir_data,"test")

# finding out the categories
labels = os.listdir(dir_train)
print("Total number of labels is ",len(labels), labels)

In [ ]:
# printing the count of each label
for l in labels:
    print(len(os.listdir(os.path.join(dir_train,l))),l)

In [ ]:
# loading the train data set
train = []
for labels_id, l in enumerate(labels):
    for file in os.listdir(os.path.join(dir_train, l)):
        train.append([dir_train+'\\{}\\{}'.format(l, file), file, labels_id, l])
train_df = pd.DataFrame(train, columns=['filepath', 'file', 'labels_id', 'labels'])

In [ ]:
train_df.sample(5)

In [ ]:
train_df.shape

In [ ]:
# reading test data set
test = []
for file in os.listdir(dir_test):
    test.append(['test\\{}'.format(file), file])
test_df = pd.DataFrame(test, columns=['filepath', 'file'])

In [ ]:
test_df.shape

In [ ]:
train_df['height']=0
train_df['width']=0
train_df['channel']=0

In [ ]:
#ximage = np.array(Image.open(train_df.filepath[0]))

In [ ]:
 for i in range(len(train_df)): 
    
    data = np.array(Image.open(train_df.filepath.values[i])) #PIL Image library
     
    train_df.loc[i,'height'] = data.shape[0]
    train_df.loc[i,'width'] = data.shape[1]
    train_df.loc[i,'channel'] = data.shape[2]

In [ ]:
data.shape

In [ ]:
train_df.shape

In [ ]:
train_df.sample(10)

In [ ]:
# images are of different sizes as can be seen from different heights and widths
# resize to same size is required

In [ ]:
train_df.height.min()

In [ ]:
train_df.height.max()

In [ ]:
train_df.width.min()

In [ ]:
train_df.height.max()

In [ ]:
import cv2
from keras.preprocessing import image
import glob

In [38]:
def load_train(path,plants,size):
    images=[]
    labels=[]
    ids=[]
    clas=[]
    for fold in plants:
        index=plants.index(fold)
        pat = os.path.join(path, fold)
        files = glob.glob(pat+'/*')
        for f1 in files:
            im=cv2.imread(f1)
            image = cv2.resize(im, (size, size), cv2.INTER_LINEAR)
            images.append(image)
            label = np.zeros(len(plants))
            label[index] = 1.0
            labels.append(label)
            #filename = os.path.basename(f1)
            #ids.append(filename)
            #clas.append(fold)
    images = np.array(images)
    labels = np.array(labels)
        #ids = np.array(ids)
        #cls = np.array(cls)
    images = images.astype(np.float32)
    images = np.multiply(images, 1.0 / 255.0)
    return images,labels

In [39]:
plants=labels
path=dir_train

In [40]:
img_train,y_train=load_train(path,plants,100)

In [41]:
img_train.shape

(4750, 100, 100, 3)

In [1]:
y_train.shape

NameError: name 'y_train' is not defined

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
x_train,x_test,y_train,y_test=train_test_split(img_train,y_train,test_size=0.25,random_state=7)

In [57]:
batch_size = 16
num_classes = 12
epochs = 10

In [58]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D

In [59]:
# Define Model
model = Sequential()

# 1st Conv Layer
model.add(Convolution2D(32, (3, 3), input_shape=(x_train.shape[1:])))
model.add(Activation('relu'))

# 2nd Conv Layer
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))

# Max Pooling
#model.add(MaxPooling2D(pool_size=(2,2)))

# Dropout
#model.add(Dropout(0.25))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))

# More Dropout
#model.add(Dropout(0.5))

# Prediction Layer
model.add(Dense(12))
model.add(Activation('softmax'))

# Loss and Optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=7, verbose=1, mode='auto')
callback_list = [early_stopping]

# fit model
model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=epochs,validation_data=(x_test, y_test),callbacks=callback_list)

C:\Users\isaur\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 3562 samples, validate on 1188 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[589824,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_3/Adam/mul_21}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_4/mul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
